In [ ]:
import our_gantt_chart as gc
import numpy as np
import pandas as pd
import datetime as dt


import matplotlib.pyplot as plt
import matplotlib.patches as patches

import matplotlib.text as text

%matplotlib notebook



In [ ]:
data = gc.read_file("gantt_chart_data_LARGE.csv")

In [ ]:
print(data)

nicknames = []

events = {}

for event in data:

    nn = event[4].astype(str)
    name = event[0].astype(str)
    type_of_work = event[5].astype(str)
    
    if nn == "":
        nn = name
    nicknames.append(nn)
    events[nn] = {}
    
    events[nn]["name"] = name
    
    date = event[1].astype(str)
    d = gc.is_date(date)
    print(d)
    start_from_other = False
    if d is False:
        d = date.split(":")[0]
        start_from_other = True

    events[nn]["start"] = d
    events[nn]["start_from_other"] = start_from_other
    events[nn]["type_of_work"] = type_of_work

    
    date = event[2].astype(str)
    d = gc.is_date(date)
    print(d)
    
    events[nn]["end"] = d
    
    duration = event[3]
    if duration=='':
        duration = None
    else:
        duration = duration.astype(float)
    events[nn]["duration"] = duration

    
print(nicknames)
print(events)

In [ ]:
date = data[0][1].astype(str)
d = gc.is_date(date)
print(d)

In [ ]:
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

colors = {"Mechanics":"r", 
           "Electronics":"b",
           "Workshop":"brown",
           "Aesthetics":"green"}


fig1 = plt.figure(figsize=(9,9))
#ax = fig1.add_subplot(111, aspect='equal')
ax = fig1.add_subplot(1,1,1)

ymax = 10000

datemin = dt.datetime(3000,1,1)
datemax = dt.datetime(1,1,1)

for i,key in enumerate(events.keys()):
    event = events[key]
    print(event)
    
    start = event["start"]
    print("start: ",start)
    if type(start) != dt.datetime:
        print("HERE")
        dependency = start
        start = events[dependency]["end"]
        print(start)
        print(type(start))
        if type(start) is not dt.timedelta and start is False:
            print(dependency)
            print(events[dependency]["start"])
            print(events[dependency]["duration"])
            start = events[dependency]["start"] + dt.timedelta(days=events[dependency]["duration"])
        print("THERE: ",start)
        event["start"] = start

    duration = event["duration"]
    if duration is None:
        duration = (event['end'] - start).days

    end = event["end"]
    if end is False:
        print(duration)
        print(start)
        end = start + dt.timedelta(days=duration)
                

    if start:
        if start<datemin:
            datemin = start
        if end>datemax:
            datemax = end
        color = colors[event["type_of_work"]]
        ax.barh(-i, duration, 0.5, start, align='center',alpha=1.0,color=color)


#fig1.autofmt_xdate()
xfmt = mdates.DateFormatter('%d-%m-%Y')
ax.xaxis.set_major_formatter(xfmt)


datemin -= dt.timedelta(days=2)
datemax += dt.timedelta(days=2)

# Put the names
for i,key in enumerate(events.keys()):
    event = events[key]
    print(event)
    name = event["name"]
    
    y = -i
    x = datemin + dt.timedelta(days=0.1)
    print("TEXT: ",name,(x,y))
    
    plt.text(x,y,name,backgroundcolor='white',bbox=(dict(facecolor='gray', alpha=0.2)),fontsize=10)

print(datemin)
print(datemax)


label_colors = []
for key in colors.keys():
    label_colors.append(colors[key])
    
#colors = ['r', 'g', 'b']
#labels = ['foo', 'bar', 'baz']
dummies = [ax.plot([], [], ls='-',lw=10, c=c)[0] for c in label_colors]        
ax.legend(dummies, colors.keys())
    
ax.set_xlim(datemin, datemax)
ax.set_ylim(-len(events)-1,1)
ax.yaxis.label.set_visible(False)

ax.set_yticklabels([])
#ax.set_yticks([])


ax.grid(axis='y')

plt.tight_layout()
plt.savefig("chart.jpg")

In [ ]:
a = dt.datetime(2017,1,1)
b = dt.datetime(2017,2,1)

c = b-a

In [ ]:
c.days

In [ ]:
len(events)

In [ ]:
text.Text